# Building a csv of all your PBs
-- by baldnate

# Get user id

In [1]:
import json
import pandas as pd
import requests

def getUserId(username):
  url = "https://www.speedrun.com/api/v1/users?name=" + username
  data = requests.get(url).json()['data']
  if len(data) == 1:
    return data[0]['id']
  else:
    raise Exception('Searched for ' + username + ', got back ' + str(len(data)) + ' entries (expected 1)') 

userid = getUserId('baldnate')

# Get PBs

In [2]:
def getPBs(userid):
  url = "https://www.speedrun.com/api/v1/users/" + userid + "/personal-bests?embed=game,category,region,platform"
  data = requests.get(url).json()['data']
  return data

pbs = getPBs(userid)
rawdf = pd.DataFrame(pbs)

# "Simple" Columns

In [3]:
runsdf = pd.DataFrame()
runsdf['place'] = rawdf['place']
runsdf['runid'] = rawdf.apply(lambda x: x.run['id'], axis=1)
runsdf['gameid'] = rawdf.apply(lambda x: x.run['game'], axis=1)
runsdf['gamename'] = rawdf.apply(lambda x: x.game['data']['names']['international'], axis=1)
runsdf['categoryname'] = rawdf.apply(lambda x: x.category['data']['name'], axis=1)
runsdf['time'] = rawdf.apply(lambda x: x.run['times']['primary_t'], axis=1)
runsdf['date'] = rawdf.apply(lambda x: x.run['date'], axis=1)
runsdf['video'] = rawdf.apply(lambda x: x.run['videos']['links'][0]['uri'], axis=1)
runsdf['comment'] = rawdf.apply(lambda x: str(x.run['comment']).replace('\n', ' ').replace('\r', ' ') , axis=1)

# Columns that need optional handling

In [4]:
def getRegion(x):
  if x.region['data'] == []:
    return None
  else:
    return x.region['data']['name']

runsdf['regionname'] = rawdf.apply(lambda x: getRegion(x), axis=1)


In [5]:
def getPlatform(x):
  if x.platform['data'] == []:
    return None
  else:
    return x.platform['data']['name']

runsdf['platformname'] = rawdf.apply(lambda x: getPlatform(x), axis=1)

# Sub-Categories

Memoized for speed and kindness.

In [6]:
varMemo = {}

In [7]:
def getVariable(variableid):
  if variableid not in varMemo:
    url = "https://www.speedrun.com/api/v1/variables/" + variableid
    response = requests.get(url)
    varMemo[variableid] = response.json()['data']
  return varMemo[variableid]

def getValue(variableid, valueid):
  var = getVariable(variableid)
  return var['values']['values'][valueid]['label']

def getSubCategories(x):
  if x.run['values'] == {}:
    return None
  else:
    vals = []
    for varid, valid in x.run['values'].items():
      if getVariable(varid)['is-subcategory']:
        vals.append(getValue(varid, valid))  
    return " -- ".join(vals)

runsdf['subcategories'] = rawdf.apply(lambda x: getSubCategories(x), axis=1)

# Dump to a csv

In [8]:
import csv
runsdf.to_csv('runs.csv', index=False, quoting=csv.QUOTE_NONNUMERIC, columns=['gamename', 'categoryname', 'subcategories', 'platformname', 'regionname', 'place', 'time', 'date','video','comment'])
print('csv exported')

csv exported


In [9]:
csvdf = pd.read_csv('runs.csv')
csvdf.head()

,gamename,categoryname,subcategories,platformname,regionname,place,time,date,video,comment
0,Super Solitaire,Free Cell,No Mouse,Super Nintendo,NaN,11,251.0,2021-08-29,https://youtu.be/FnyM3XdZ64A,Played on an SNS-001 with SD2SNES and gamepad.
1,Super Solitaire,Golf,Dead King -- No Mouse,Super Nintendo,NaN,4,72.0,2021-08-29,https://youtu.be/EnAiEkuWK8Q,Played on an SNS-001 with SD2SNES and gamepad.
2,Super Solitaire,Klondike,1 Flip -- No Mouse,Super Nintendo,NaN,7,140.0,2021-08-28,https://youtu.be/7pADwt4V-Js,Played on an SNS-001 with SD2SNES and gamepad.
3,Super Solitaire,Poker,No Mouse,Super Nintendo,NaN,4,0.1,2021-08-28,https://youtu.be/3dP4e1yJXLs,Played on an SNS-001 with SD2SNES and gamepad.
4,Alien Crush,10 Million Points,NaN,TurboGrafx-16/PC Engine,JPN / NTSC,1,498.0,2021-07-23,https://www.youtube.com/watch?v=rE6Gz2wrWYY,Played on a TG-16 with TBED.
